## Solving the environment

PROBLEM: it's a bit confusing that you can't use PPO. As the first application, we might want something where PPO just works. 
hmm, ok so here are the PPO tuned examples:
https://github.com/ray-project/ray/tree/master/rllib/tuned_examples/ppo

even for these, though, there are tons of hyperparameters set

In [4]:
import ray  # .. of course
# from ray import serve
# from ray import tune

from collections import OrderedDict
import gym  # RL environments and action/observation spaces
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
from pprint import pprint
import re
import recsim  # google's RecSim package.
import requests
from scipy.interpolate import make_interp_spline, BSpline
from scipy.stats import linregress, sem
from starlette.requests import Request
import tree  # dm_tree

ModuleNotFoundError: No module named 'recsim'

In [1]:
from ray.rllib.agents.slateq import SlateQTrainer

slateq_config = {
    "env": "modified_lts",
    "env_config": {
        "num_candidates": 20,  # MultiDiscrete([20, 20]) -> no flattening necessary (see `convert_to_discrete_action_space=False` below)
        "slate_size": 2,
        "resample_documents": True,
        "wrap_for_bandits": False,  # SlateQ != Bandit (will keep "doc" key, instead of "items")
        "convert_to_discrete_action_space": False,  # SlateQ handles MultiDiscrete action spaces (slate recommendations).
    },
    # Setup exploratory behavior: Implemented as "epsilon greedy" strategy:
    # Act randomly `e` percent of the time; `e` gets reduced from 1.0 to almost 0.0 over
    # the course of `epsilon_timesteps`.
    "exploration_config": {
        #"warmup_timesteps": 20000,  # default
        "epsilon_timesteps": tune.grid_search([40000, 2000, 3000]),  # default: 250000
    },
    #"learning_starts": 20000,  # default
    "target_network_update_freq": 3200,

    # Report rewards as smoothed mean over this many episodes.
    "metrics_num_episodes_for_smoothing": 200,
}

# Instantiate the Trainer object using the exact same config as in our Bandit experiment above.
slateq_trainer = SlateQTrainer(config=slateq_config)
slateq_trainer

NameError: name 'tune' is not defined

In [2]:
lts_20_2_env = LTSWithStrongerDissatisfactionEffect(LongTermSatisfactionRecSimEnv(config=slateq_config["env_config"]))

obs = lts_20_2_env.reset()
done = False
while not done:
    action = slateq_trainer.compute_single_action(input_dict={"obs": obs})
    feat_value_of_action = obs["doc"][str(action[0])][0]
    max_feat_action = np.argmax([value for _, value in obs["doc"].items()])
    max_choc_feat = obs['doc'][str(max_feat_action)][0]
    # Print out the picked document's feature value and compare that to the highest possible feature value.
    print(f"action's feature value={feat_value_of_action} max-choc-feature={max_choc_feat}")
    
    obs, r, done, _ = lts_20_2_env.step(action)

NameError: name 'LTSWithStrongerDissatisfactionEffect' is not defined